In [3]:
import fitz  # PyMuPDF
import os

# Create output directory
output_dir = "D:/home/upload/extracted_images"
os.makedirs(output_dir, exist_ok=True)

doc = fitz.open("D:/home/upload/test.pdf")
for page in doc:
    image_list = page.get_images()
    for img_index, img in enumerate(image_list):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        pix.save(f"{output_dir}/image_{page.number}_{img_index}.png")
        pix = None
doc.close()
print(f"Images saved to: {output_dir}")

Images saved to: D:/home/upload/extracted_images


In [4]:
import pdfplumber
from PIL import Image
import os

# Create output directory
output_dir = "D:/home/upload/images"
os.makedirs(output_dir, exist_ok=True)

with pdfplumber.open("D:/home/upload/test.pdf") as pdf:
    for page_num, page in enumerate(pdf.pages):
        for img_num, img in enumerate(page.images):
            # Get image bounding box
            x0, top, x1, bottom = img["x0"], img["top"], img["x1"], img["bottom"]
            
            # Crop the image from the page
            cropped_page = page.crop((x0, top, x1, bottom))
            
            # Convert to PIL Image
            pil_image = cropped_page.to_image(resolution=150).original
            
            # Save the image to directory
            output_path = f"{output_dir}/page_{page_num + 1}_image_{img_num + 1}.png"
            pil_image.save(output_path)
            print(f"Saved: {output_path}")

print(f"All images saved to: {output_dir}")

Saved: D:/home/upload/images/page_1_image_1.png
Saved: D:/home/upload/images/page_2_image_1.png
Saved: D:/home/upload/images/page_2_image_2.png
Saved: D:/home/upload/images/page_2_image_3.png
Saved: D:/home/upload/images/page_2_image_4.png
Saved: D:/home/upload/images/page_2_image_5.png
Saved: D:/home/upload/images/page_2_image_6.png
Saved: D:/home/upload/images/page_2_image_7.png
Saved: D:/home/upload/images/page_2_image_8.png
Saved: D:/home/upload/images/page_2_image_9.png
Saved: D:/home/upload/images/page_3_image_1.png
Saved: D:/home/upload/images/page_3_image_2.png
Saved: D:/home/upload/images/page_3_image_3.png
Saved: D:/home/upload/images/page_3_image_4.png
Saved: D:/home/upload/images/page_3_image_5.png
Saved: D:/home/upload/images/page_3_image_6.png
Saved: D:/home/upload/images/page_3_image_7.png
Saved: D:/home/upload/images/page_3_image_8.png
Saved: D:/home/upload/images/page_3_image_9.png
Saved: D:/home/upload/images/page_3_image_10.png
Saved: D:/home/upload/images/page_3_ima

In [ ]:
import fitz  # PyMuPDF
import json

def extract_tagged_images(pdf_path):
    doc = fitz.open(pdf_path)
    images_with_tags = []
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        
        # Get all images with full metadata
        image_list = page.get_images(full=True)
        
        for img_idx, img in enumerate(image_list):
            xref = img[0]  # Image reference
            base_image = doc.extract_image(xref)
            
            # Extract image data
            image_bytes = base_image["image"]
            img_metadata = {
                "xref": xref,
                "width": base_image["width"],
                "height": base_image["height"],
                "ext": base_image["ext"],
                "page": page_num,
                "bbox": page.get_image_bbox(img).to_dict()  # Spatial position
            }
            
            # **CRITICAL**: Extract associated text/tags near image
            bbox = page.get_image_bbox(img)
            nearby_text = page.get_text("dict", clip=fitz.Rect(
                bbox.x0-50, bbox.y0-50, bbox.x1+50, bbox.y1+50
            ))
            
            # Find captions/headings near image
            caption = extract_caption_near_image(nearby_text)
            
            images_with_tags.append({
                **img_metadata,
                "caption": caption,
                "image_data": image_bytes
            })
    
    doc.close()
    return images_with_tags

def extract_caption_near_image(text_dict):
    """Extracts property address/caption near image"""
    for block in text_dict["blocks"]:
        if "lines" in block:
            for line in block["lines"]:
                text = "".join(span["text"] for span in line["spans"])
                if any(keyword in text.lower() for keyword in 
                       ["w 23rd", "avenue", "street", "$", "beds"]):
                    return text.strip()
    return ""



Processing: 3436 W 23rd Ave CMA_quick.pdf
  Saved: 3436 W 23rd Ave CMA_quick_page1_img1.png
  Caption: Researched and prepared by Subject Property
Nathan Adams 3436 W 23rd Avenue
Prepared exclusively for Denver, Colorado
Nate Adams
80211
Prepared on
December 26, 2025
Nathan Adams
REDT LLC
4260 E EVANS AVE
DENVER, CO 80222
720-255-4010
nathan@redTHomes.com
http://www.realestatedenverteam.com
Copyright: 2025 All rights reserved.
This is a broker price opinion or comparative market analysis and should not be considered an appraisal or opinion of value.
In making any decision that relies upon my work, you should know that I have not followed the guidelines for development of
an appraisal or analysis contained in the Uniform Standards of Professional Appraisal Practice of the Appraisal Foundation.
  Saved: 3436 W 23rd Ave CMA_quick_page1_img2.png
  Caption: No caption found
  Saved: 3436 W 23rd Ave CMA_quick_page1_img3.png
  Caption: RE
42
DE
72
na
htt
This is a broker p
In making any de
a

In [ ]:
import pdfplumber
import os
import json

# Configuration
pdf_path = "D:/home/upload/3436 W 23rd Ave CMA_quick.pdf"
output_dir = "D:/home/upload/captions"
os.makedirs(output_dir, exist_ok=True)

results = []

with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages):
        images = page.images
        
        if not images:
            continue
        
        # Sort images by position (top to bottom, left to right)
        sorted_images = sorted(images, key=lambda x: (x['top'], x['x0']))
        
        for img_num, img in enumerate(sorted_images):
            # Image bounding box
            img_bbox = (img['x0'], img['top'], img['x1'], img['bottom'])
            
            # Caption region: below the image (same width, extend down ~30 pixels)
            caption_bbox = (
                img['x0'],
                img['bottom'],  # Start at bottom of image
                img['x1'],
                min(page.height, img['bottom'] + 20)  # Extend 20px below
            )
            print(caption_bbox)
            # Extract caption text
            caption_region = page.crop(caption_bbox)
            print(caption_region)
            caption = caption_region.extract_text() or ""
            caption = caption.strip()
            
            # Extract and save image
            try:
                img_obj = page.crop(img_bbox).to_image(resolution=200)
                img_filename = f"page{page_num + 1}_img{img_num + 1}.png"
                img_save_path = os.path.join(output_dir, img_filename)
                img_obj.save(img_save_path)
                
                results.append({
                    "page": page_num + 1,
                    "image": img_filename,
                    "caption": caption
                })
                
                print(f"Page {page_num + 1}, Image {img_num + 1}: {caption}...")
            except Exception as e:
                print(f"Error on page {page_num + 1}, image {img_num + 1}: {e}")

# Save captions to JSON
with open(os.path.join(output_dir, "image_captions.json"), "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# Save captions to text file
with open(os.path.join(output_dir, "image_captions.txt"), "w", encoding="utf-8") as f:
    for item in results:
        f.write(f"Page {item['page']} - {item['image']}\n")
        f.write(f"Caption: {item['caption']}\n")
        f.write("-" * 50 + "\n")

print(f"\n✅ Extracted {len(results)} images with captions")
print(f"📁 Output: {output_dir}")

(14.4, 751.6, 591.3, 771.6)
<Page:1>
Page 1, Image 1: ...
(277.65, 329.7, 537.95, 349.7)
<Page:1>
Page 1, Image 2: ...
(32.4, 669.1, 100.69999999999999, 689.1)
<Page:1>
Page 1, Image 3: ...
(26.4, 737.4, 101.4, 757.4)
<Page:1>
Page 1, Image 4: ...
(68.4, 82.39999999999998, 576.15, 102.39999999999998)
<Page:2>
Page 2, Image 1: Subject Property: 3436 W 23rd Avenue, Denver 80211 December 26, 2025...
(68.4, 82.39999999999998, 576.15, 102.39999999999998)
<Page:3>
Page 3, Image 1: Subject Property: 3436 W 23rd Avenue , Denver, 80211 December 26, 2025...
(52.65, 258.15, 214.70000000000002, 278.15)
<Page:3>
Page 3, Image 2: 3436 W 23rd Avenue...
(224.55, 258.65, 386.3, 278.65)
<Page:3>
Page 3, Image 3: 2280 W 42nd Avenue...
(403.15, 258.65, 564.9, 278.65)
<Page:3>
Page 3, Image 4: 3424 W 31st Avenue...
(68.4, 82.39999999999998, 576.15, 102.39999999999998)
<Page:4>
Page 4, Image 1: Subject Property: 3436 W 23rd Avenue , Denver, 80211 December 26, 2025...
(52.65, 258.15, 214.70000000000002, 278.

In [2]:
%pip install --upgrade --force-reinstall youtube-transcript-api


  Using cached youtube_transcript_api-1.2.3-py3-none-any.whl.metadata (24 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
Using cached youtube_transcript_api-1.2.3-py3-none-any.whl (485 kB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached certifi-2026.1.4-py3-none-any.whl (152 kB)
Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl (107 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached urllib3-2.6.3-py3-none-any.whl (131 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.6.3
    Uninstalling urlli

  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
print(dir(YouTubeTranscriptApi))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fetch', 'list']


In [8]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "rmvDxxNubIg"

api = YouTubeTranscriptApi()

# List available transcripts
transcript_list = api.list(video_id)

# Pick English if available, otherwise first transcript
try:
    transcript = transcript_list.find_transcript(['en'])
except:
    transcript = transcript_list[0]

entries = transcript.fetch()

# Full text
full_text = " ".join(e.text for e in entries)
print(full_text)

# With timestamps
for e in entries[:10]:
    print(f"{e.start:.2f}s: {e.text}")


[music] Hi everybody. How y'all doing? >> It's exciting. I'm Dex. Uh, as they did in the great intro, I've been hacking on agents for a while. Um, our talk 12 factor agents at AI engineer in June was one of the top talks of all time. Uh, I think top eight or something. One of the best ones from from AI engineer in June. May or may not have said something about context engineering. Um, why am I here today? What am I here to talk about? Um, I want to talk about one of my favorite talks from AI engineer in June. And I know we all got the update from Eigor yesterday, but they wouldn't let me change my slides. So, this is going to be about what Eigor talked about in June. uh basically that they surveyed a 100,000 developers across all company sizes and they found that most of the time you use AI for software engineering you're doing a lot of rework a lot of codebase churn uh and it doesn't really work well for complex tasks brownfield code bases um and you can see in the chart basically you